<a href="https://colab.research.google.com/github/solomontessema/building-agentic-ai/blob/main/notebooks/6_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Define schema extraction logic</h1></td>
  </tr>
</table>

 ## From Questions to Queries: GPT as a SQL Generator

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
import sqlite3
import random
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

# Connect to (or create) a database file
db = sqlite3.connect("data.db")

cursor = db.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    signup_date TEXT,
    is_active INTEGER
)
""")

db.commit()

cursor = db.cursor()

sample_users = [
    (1, "Alice Johnson", "alice@example.com", "2023-02-15", 1),
    (2, "Bob Smith", "bob@example.com", "2022-11-03", 0),
    (3, "Charlie Zhang", "charlie@example.com", "2023-07-21", 1),
    (4, "Diana Lopez", "diana@example.com", "2021-05-10", 0)
]

cursor.executemany("""
INSERT INTO users (id, name, email, signup_date, is_active)
VALUES (?, ?, ?, ?, ?)
""", sample_users)


db.close()



## Create a database connector.py


In [ ]:

class SQLiteConnector:
    def __init__(self, db_path="data.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def list_tables(self):
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        return [row[0] for row in self.cursor.fetchall()]

    def describe_table(self, table_name):
        self.cursor.execute(f"PRAGMA table_info({table_name});")
        return self.cursor.fetchall()

    def get_schema_summary(self):
        summary = []
        for table in self.list_tables():
            cols = self.describe_table(table)
            formatted = f"Table: {table} Columns: {[col[1] for col in cols]}"
            summary.append(formatted)
        return "\n".join(summary)

if __name__ == "__main__":
    db = SQLiteConnector()
    print(db.get_schema_summary())

In [ ]:
from dotenv import load_dotenv
load_dotenv()

connector = SQLiteConnector()
schema_summary = connector.get_schema_summary()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    Given the database schema:
    {schema}

    Write a SQLite SQL query to answer the question:
    {question}
    """
)

sql_runnable = prompt | llm | StrOutputParser()

def generate_sql(question: str):
    return sql_runnable.invoke({"schema": schema_summary, "question": question})
# Step 2: Test SQL Generation
print(generate_sql("How many users signed up in 2023?"))
